# Bibliotecas

In [1]:
import os
import pandas as pd
import numpy as np
import scipy as sp
from scipy import signal
import ml_time_series as mls
import timeit
from datetime import datetime
from sklearn.externals import joblib
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn import preprocessing
from sklearn import utils

%matplotlib notebook

In [2]:
def Stacker(files_,t):
    
    #Lê dados do arquivo e remove janela
    Xc = np.load('DB_INTENC/preproc/'+files_[0]+'.npy')
    Xc = Xc[:-49]
    
    #Aplica Hstack nos dados
    for j in range(1,len(files_)):
        Xci = np.load('DB_INTENC/preproc/'+files_[j]+'.npy')
        Xc = np.hstack((Xc.reshape(Xc.shape),Xci.reshape(Xci.shape)))
        
    #Labeling the PKS level    
    #C = (np.ones(len(Xc))*l).reshape((len(Xc),1))
    #Xc = np.hstack((Xc.reshape(Xc.shape),C.reshape((len(Xc),1))))    
    
    # Salving in file on the folder <DB_INTENC/preproc/labeled>
    np.save('./DB_INTENC/preproc/labeled/'+t+'_hs', Xc, allow_pickle=False)
    DATA_hs.append(t+'_hs.npy')

In [15]:
files_ = [['p1_std', 'p1_rms', 'p1_filt'],['p2_std', 'p2_rms', 'p2_filt'],['p3_std', 'p3_rms', 'p3_filt'],
          ['p5_std', 'p5_rms', 'p5_filt'],['c1_std', 'c1_rms', 'c1_filt'],['c3_std', 'c3_rms', 'c3_filt'],
          ['p4_std', 'p4_rms', 'p4_filt']]
DATA_hs = []

In [4]:
files_ = [['p4_std', 'p4_rms', 'p4_filt']]
DATA_hs = []

In [16]:
#Grupo de validação
Stacker(files_[0],files_[0][0][:2])

In [17]:
#Grupo total
for i in range(len(files_)):
    Stacker(files_[i],files_[i][0][:2])

In [18]:
DATA_hs

['p1_hs.npy',
 'p1_hs.npy',
 'p2_hs.npy',
 'p3_hs.npy',
 'p5_hs.npy',
 'c1_hs.npy',
 'c3_hs.npy',
 'p4_hs.npy']

In [19]:
Xc = np.load('DB_INTENC/preproc/labeled/p1_hs.npy')

In [20]:
dfn = pd.DataFrame(data=Xc)
dfn.head()

,0,1,2,3,4,5,6,7,8,9,10,11
0,-0.079685,0.009403,-0.013066,-0.004118,44.537192,7.559818,18.959109,25.976221,0.045622,-0.071708,0.928207,0.316825
1,-0.021022,0.003217,-0.013066,0.021096,44.625654,7.583042,19.512969,25.987871,-0.653900,-0.028405,1.233691,0.323795
2,0.078706,0.023837,-0.013066,0.042108,44.652662,7.607774,20.306241,26.098226,-1.344409,-0.094986,1.645009,0.363447
3,0.078706,0.001155,-0.001691,0.050512,44.692366,7.280775,21.180114,26.114275,-1.876668,-0.237642,2.238290,0.454788
4,0.131503,0.007341,-0.009275,0.012691,45.297386,7.296239,22.324117,26.340057,-2.066876,-0.336827,3.042093,0.596534


In [21]:
Xc.shape

(53951, 12)

In [22]:
def proccess_data(x,DATA,NUM_SAMPLES,LABEL):
    
    Xc = np.load("DB_INTENC/preproc/labeled/"+x)
    print '\nXc initial shape ', Xc.shape
    
    #Principal component analysis
    pca = PCA(n_components=3)
    pca.fit(Xc)
    Xc = pca.transform(Xc)
    print pca.explained_variance_ratio_
    
    print 'Xc PCA shape ', Xc.shape
    
    #Create temporal serie
    #Xc = mls.generate_envelope(Xc, NUM_SAMPLES)
    #print 'Xc temporal-serie shape ', Xc.shape
    
    #Labeling the PKS level    
    C = (np.ones(len(Xc))*LABEL).reshape((len(Xc),1))
    Xc = np.hstack((Xc.reshape(Xc.shape),C.reshape((len(Xc),1)))) 
    print 'Xc labeled shape ', Xc.shape
    
    # Salving in file on the folder <DB_INTENC/preproc/labeled>
    np.save('./DB_INTENC/preproc/labeled/'+x[:2]+'_tsl', Xc, allow_pickle=False)
    print '\n'+x[:2]+'_tsl'
    
    DATA.append(x[:2]+'_tsl.npy')

In [23]:
DATA_hs

['p1_hs.npy',
 'p1_hs.npy',
 'p2_hs.npy',
 'p3_hs.npy',
 'p5_hs.npy',
 'c1_hs.npy',
 'c3_hs.npy',
 'p4_hs.npy']

---

In [54]:
#Validação
start = timeit.default_timer()

NUM_SAMPLES = 500
l = [1]
DATA = []

for x, y in zip(DATA_hs, l):
    proccess_data(x,DATA,NUM_SAMPLES,y)
    
stop = timeit.default_timer()
ti = stop - start 
print ('Tempo total para o envelopamento: %.2f min' % (ti/60))


Xc initial shape  (29951, 12)
[ 0.82172641  0.12642296  0.02960795]
Xc PCA shape  (29951, 3)
Xc temporal-serie shape  (29452, 1500)
Xc labeled shape  (29452, 1501)

p4_hs.npy_tsl
Tempo total para o envelopamento: 27.59 min


In [55]:
DATA

['p4_hs.npy_tsl.npy']

In [56]:
Xc = np.load('DB_INTENC/preproc/labeled/p4_hs.npy_tsl.npy')
dfn = pd.DataFrame(data=Xc)
dfn.head()

,0,1,2,3,4,5,6,7,8,9,...,1491,1492,1493,1494,1495,1496,1497,1498,1499,1500
0,-24.899834,-12.359730,3.314693,-25.308322,-12.263494,3.355825,-25.753507,-12.189007,3.461927,-26.464323,...,-24.605918,-8.172779,8.611135,-24.858806,-8.181450,8.651859,-25.145841,-8.138746,8.924400,1.0
1,-25.308322,-12.263494,3.355825,-25.753507,-12.189007,3.461927,-26.464323,-12.072625,3.473236,-26.833126,...,-24.858806,-8.181450,8.651859,-25.145841,-8.138746,8.924400,-25.431671,-8.098234,9.147232,1.0
2,-25.753507,-12.189007,3.461927,-26.464323,-12.072625,3.473236,-26.833126,-11.989849,3.397637,-27.248365,...,-25.145841,-8.138746,8.924400,-25.431671,-8.098234,9.147232,-25.686794,-7.915669,9.248339,1.0
3,-26.464323,-12.072625,3.473236,-26.833126,-11.989849,3.397637,-27.248365,-11.907640,3.321180,-27.878416,...,-25.431671,-8.098234,9.147232,-25.686794,-7.915669,9.248339,-25.836654,-7.596296,9.085521,1.0
4,-26.833126,-11.989849,3.397637,-27.248365,-11.907640,3.321180,-27.878416,-11.829128,3.291034,-28.623097,...,-25.686794,-7.915669,9.248339,-25.836654,-7.596296,9.085521,-25.639430,-7.276366,8.819659,1.0


---

In [24]:
l = [2, 15, 4, 1, 0, 0, 15]
for x, y in zip(DATA_hs, l):
    print x[:2],y

p1 2
p1 15
p2 4
p3 1
p5 0
c1 0
c3 15


In [25]:
#Grupo total
start = timeit.default_timer()

NUM_SAMPLES = 500
l = [2, 15, 4, 1, 0, 0, 15]
DATA = []

for x, y in zip(DATA_hs, l):
    proccess_data(x,DATA,NUM_SAMPLES,y)
    
stop = timeit.default_timer()
ti = stop - start 
print ('Tempo total para o envelopamento: %.2f min' % (ti/60))


Xc initial shape  (53951, 12)
[ 0.61593794  0.29817543  0.02675621]
Xc PCA shape  (53951, 3)
Xc labeled shape  (53951, 4)

p1_tsl

Xc initial shape  (53951, 12)
[ 0.61593794  0.29817543  0.02675621]
Xc PCA shape  (53951, 3)
Xc labeled shape  (53951, 4)

p1_tsl

Xc initial shape  (25951, 12)
[ 0.74124299  0.17419237  0.0367697 ]
Xc PCA shape  (25951, 3)
Xc labeled shape  (25951, 4)

p2_tsl

Xc initial shape  (61951, 12)
[ 0.71992864  0.18989168  0.06194009]
Xc PCA shape  (61951, 3)
Xc labeled shape  (61951, 4)

p3_tsl

Xc initial shape  (23951, 12)
[ 0.64202733  0.25305677  0.08240487]
Xc PCA shape  (23951, 3)
Xc labeled shape  (23951, 4)

p5_tsl

Xc initial shape  (17951, 12)
[ 0.64981489  0.26427588  0.06478059]
Xc PCA shape  (17951, 3)
Xc labeled shape  (17951, 4)

c1_tsl

Xc initial shape  (19951, 12)
[ 0.75157388  0.12123081  0.05738713]
Xc PCA shape  (19951, 3)
Xc labeled shape  (19951, 4)

c3_tsl
Tempo total para o envelopamento: 0.01 min


In [13]:
#Não foram incluidas as amostras p4 e c3, serão utilizadas para validação!

In [26]:
DATA[1:-2]

['p1_tsl.npy', 'p2_tsl.npy', 'p3_tsl.npy', 'p5_tsl.npy']

---

## Shape data

In [27]:
Xc = np.load("DB_INTENC/preproc/labeled/"+DATA[0])
for i in DATA[1:-2]:
    Xc = np.vstack((Xc,np.load('DB_INTENC/preproc/labeled/'+i))) 

### Split tipo 1

In [242]:
#Dividindo os dados em sets de treino, test e validação
dfn = pd.DataFrame(data=Xc)
train, validate, test = np.split(dfn.sample(frac=1), [int(.6*len(dfn)), int(.8*len(dfn))])

In [244]:
np.save('./DB_INTENC/preproc/labeled/train', train, allow_pickle=False)
np.save('./DB_INTENC/preproc/labeled/validate', validate, allow_pickle=False)
np.save('./DB_INTENC/preproc/labeled/test', test, allow_pickle=False)

In [14]:
Xc_train = np.load("DB_INTENC/preproc/labeled/train.npy")

X_train = Xc_train[:,0:(Xc_train.shape[1]-1)]

yz = Xc_train[:,[(Xc_train.shape[1]-1)]]
y_train = np.array([])
for i in range(len(yz)):
    y_train = np.hstack((y_train,yz[i]))
    
lab_enc = preprocessing.LabelEncoder()
y_train = lab_enc.fit_transform(y_train)

X_train.shape, y_train.shape

((98284, 1500), (98284,))

In [77]:
#Xc_validate = np.load("DB_INTENC/preproc/labeled/validate.npy")
Xc_validate = Xc

X_validate = Xc_validate[:,0:(Xc_validate.shape[1]-1)]

yz = Xc_validate[:,[(Xc_validate.shape[1]-1)]]
y_validate = np.array([])
for i in range(len(yz)):
    y_validate = np.hstack((y_validate,yz[i]))
    
#lab_enc = preprocessing.LabelEncoder()
#y_validate = lab_enc.fit_transform(y_validate)

X_validate.shape, y_validate.shape

((29452, 1500), (29452,))

In [24]:
Xc_test= np.load("DB_INTENC/preproc/labeled/validate.npy")

X_test = Xc_test[:,0:(Xc_test.shape[1]-1)]

yz = Xc_test[:,[(Xc_test.shape[1]-1)]]
y_test = np.array([])
for i in range(len(yz)):
    y_test = np.hstack((y_test,yz[i]))
    
lab_enc = preprocessing.LabelEncoder()
y_test = lab_enc.fit_transform(y_test)

X_test.shape, y_test.shape

((32762, 1500), (32762,))

### Split tipo 2

In [28]:
#Separando set de dados X, e set de labels y - Validação DATA[5] e DATA[6]
Xc = np.load("DB_INTENC/preproc/labeled/"+DATA[5])
    
X_validate = Xc[:,0:(Xc.shape[1]-1)]

yz = Xc[:,[(Xc.shape[1]-1)]]
y_validate = np.array([])
for i in range(len(yz)):
    y_validate = np.hstack((y_validate,yz[i]))
    
X_validate.shape, y_validate.shape

((17951, 3), (17951,))

In [29]:
#Separando set de dados X, e set de labels y - Treino e Teste
Xc = np.load("DB_INTENC/preproc/labeled/"+DATA[0])
for i in DATA[1:-2]:
    Xc = np.vstack((Xc,np.load('DB_INTENC/preproc/labeled/'+i))) 
    
X = Xc[:,0:(Xc.shape[1]-1)]

yz = Xc[:,[(Xc.shape[1]-1)]]
y = np.array([])
for i in range(len(yz)):
    y = np.hstack((y,yz[i]))
    
X.shape, y.shape

((219755, 3), (219755,))

In [30]:
np.unique(y)

array([  0.,   1.,   4.,  15.])

In [31]:
#Dividindo os dados em sets de treino e test
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=1)

X_test, X_validate, y_test, y_validate = train_test_split(X, y, test_size=0.5, random_state=1)

In [32]:
#Aplica normalizador P.S.: Verificar necessidade pois os dados já foram normalizados anteriormente
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
sc.fit(X_train)
X_train_std = sc.transform(X_train)
X_test_std = sc.transform(X_test)
X_validate_std = sc.transform(X_validate)

print X_train_std.shape, X_test_std.shape, X_validate_std.shape, y_train.shape, y_test.shape, y_validate.shape

(153828, 3) (109877, 3) (109878, 3) (153828,) (109877,) (109878,)


In [33]:
X_validate_std = sc.transform(X_validate)

X_validate_std.shape, y_validate.shape

((109878, 3), (109878,))

---

## Classifiers

### Random Forest

In [34]:
#Training and test
from sklearn.ensemble import RandomForestClassifier
rfc = RandomForestClassifier(n_estimators=100, n_jobs = 4)

print 'Training time'
%time rfc = rfc.fit(X_train_std, y_train)

print''

print 'Predict time'
%time y_pred = rfc.predict(X_test_std)

print''

from sklearn.metrics import accuracy_score
print ('ClassifyRF accuracy:---------->%.2f %%' % (accuracy_score(y_test, y_pred)*100))

Training time
CPU times: user 47.7 s, sys: 264 ms, total: 47.9 s
Wall time: 12.4 s

Predict time
CPU times: user 5.66 s, sys: 304 ms, total: 5.96 s
Wall time: 1.68 s

ClassifyRF accuracy:---------->100.00 %


In [67]:
#Salva a rede treinada para um arquivo
joblib.dump(rfc, 'DB_INTENC/preproc/labeled/rfc_split2.pkl') 

['DB_INTENC/preproc/labeled/rfc_split2.pkl']

In [26]:
rfc = joblib.load('DB_INTENC/preproc/labeled/rfc.pkl')

In [35]:
#Validation
print 'Predict time'
%time y_pred = rfc.predict(X_validate)

print''

from sklearn.metrics import accuracy_score

result = ('ClassifyRF accuracy:---------->%.2f %%' % (accuracy_score(y_pred, y_validate)*100))

print result

Predict time
CPU times: user 1.38 s, sys: 168 ms, total: 1.55 s
Wall time: 587 ms

ClassifyRF accuracy:---------->25.66 %


In [36]:
#Validation
print 'Predict time'
%time y_pred = rfc.predict(X_validate_std)

print''

from sklearn.metrics import accuracy_score

result = ('ClassifyRF accuracy:---------->%.2f %%' % (accuracy_score(y_pred, y_validate)*100))

print result

Predict time
CPU times: user 5.54 s, sys: 204 ms, total: 5.75 s
Wall time: 1.56 s

ClassifyRF accuracy:---------->91.00 %


---

## Support Vector Machine

In [ ]:
from sklearn import svm, datasets

print 'Processing time SVM'
%time svm = svm.SVC(kernel='rbf', gamma=0.2, C=6, decision_function_shape='ovr').fit(X_train_std, y_train)

print ''

from sklearn.metrics import accuracy_score

print 'Predict time'  
%time y_pred = svm.predict(X_test_std)
from sklearn.metrics import accuracy_score
print ('SVM-RBF accuracy:---------->%.2f %%' % (accuracy_score(y_test, y_pred)*100))

Processing time SVM
